<a href="https://colab.research.google.com/github/vlordier/colabs/blob/main/Simple_diarizer_youtube5(JSON_Included)_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JUST YOU NEED TO UPLOAD DATASET (/CONTENT/DIARIZERDATASET1)AND RUN ALL

# **Install Dependencies**

In [ ]:
!pip install simple-diarizer
!pip install matplotlib==3.1.3

     |████████████████████████████████| 791 kB 15.6 MB/s 
     |████████████████████████████████| 97 kB 7.4 MB/s 
     |████████████████████████████████| 1.9 MB 44.8 MB/s 
     |████████████████████████████████| 11.2 MB 41.5 MB/s 
     |████████████████████████████████| 2.9 MB 33.4 MB/s 
     |████████████████████████████████| 408 kB 48.7 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 380 kB 51.9 MB/s 
     |████████████████████████████████| 895 kB 21.4 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 45.7 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.5 MB/s eta 0:00:33tcmalloc: large alloc 1147494400 bytes == 0x55a08e906000 @  0x7f0cdfd05615 0x55a0544533bc 0x55a05453418a 0x55a0544561cd 0x55a054548b3d 0x55a0544ca458 0x55a0544c502f 0x55a054457aba 0x55a0544ca2c0 0x55a0544c502f 0x55a054457aba 0x55a0544c6cd4 0x55a054549986 0x55a0544c6350 0x55a054549986 0x55a0544c635

     |████████████████████████████████| 13.1 MB 13.2 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.1
    Uninstalling matplotlib-3.5.1:
      Successfully uninstalled matplotlib-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simple-diarizer 0.0.9 requires matplotlib==3.5.1, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from simple_diarizer.diarizer import Diarizer
from simple_diarizer.utils import (check_wav_16khz_mono, convert_wavfile, parse_ttml, 
                                   waveplot, combined_waveplot, waveplot_perspeaker, get_youtube_id, 
                                   download_youtube_wav, download_youtube_ttml)

import os
import tempfile
from pprint import pprint

import matplotlib.pyplot as plt
import soundfile as sf

from IPython.display import Audio, display, HTML
from tqdm.autonotebook import tqdm
import pandas as pd
import pandas as pd
import traceback
os.mkdir('dialogues')
os.mkdir('processed')
os.mkdir('orignal_caption')
os.mkdir('orignal_processed')

## **Speaker Diarizer Inferences**

In [ ]:
df=pd.read_csv("/content/diarizer_dataset1.csv")
df_links=df.dropna()

In [ ]:
links=[]
beg=[]
end=[]
for i in range(len(df)):
  if ((str((df.iloc[i,1])).strip()=="Beginning") | ((str(df.iloc[i,1])).strip()=="beginning")):
    beg.append(df.iloc[i,2])
  if ((str(df.iloc[i,1]).strip()=="End")or ((str(df.iloc[i,1])).strip()=="end")):  
    end.append(df.iloc[i,2]) 


In [ ]:
import datetime
import time
from datetime import timedelta
def seconds(t):
  if(len(t)<=5): 
    if(len(t)==5):

      x = time.strptime('00:'+t,'%H:%M:%S')
      sec=datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()
  
    if(len(t)==4):
      x = time.strptime('00:0'+t,'%H:%M:%S')
      sec=datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()
  else:
    if(len(t)==8):
      if(int(t[0:2])>=24):
        print("hours are greater than 24") 
        
        sec=int(t[0:2])*3600
      else:
        print(t[0:2])
        x = time.strptime(t,'%H:%M:%S')
        sec=datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()
    if(len(t)==7):
      x = time.strptime('0'+t,'%H:%M:%S')
      sec=datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()
      
   
  return int(sec)
seconds(beg[1])

2

In [ ]:
import pandas as pd
import traceback
diar = Diarizer(
        embed_model='ecapa', # supported types: ['xvec', 'ecapa']
        cluster_method='sc', # supported types: ['ahc', 'sc']
        window=1.5, # size of window to extract embeddings (in seconds)
        period=0.75 # hop of window (in seconds)
    )
def speaker_diarizer(link_id,i,s,e):
  YOUTUBE_link = "https://www.youtube.com/embed/"+link_id +'?start='+str(int(s))+'&end='+str(int(e)) # The YouTube ID of the video you wish to diarize
  print(YOUTUBE_link)
  #https://www.youtube.com/embed/MUtRnMMOTmo?start=36&end=65
  NUM_SPEAKERS = 2 # The number of speakers
  try:
    outih_segments, outih_worded_segments, outih_wav = diar.diarize_youtube(YOUTUBE_link, 
                                                                        num_speakers=None, silence_tolerance=0.2,
                                                                        threshold=3, 
                                                                        outfolder='./outiih')
    !youtube-dl --write-auto-sub --sub-lang en --sub-format ttml --skip-download -o "%(id)s.%(ext)s" $link_id
    text_segments = parse_ttml('./{}.en.ttml'.format(link_id))
    yt_worded_segments = diar.match_diarization_to_transcript(outih_segments, text_segments)
    diar.nice_text_output(yt_worded_segments, '/content/dialogues/yt_test'+str(i)+".txt")
    diar.nice_text_output(yt_worded_segments, '/content/orignal/orignal'+str(i)+".txt")
  except Exception as error:
    traceback.print_exc()
    print("there is no caption in video")
link_list=[]
for i in range(len(df_links)):
  #print(i)
  link_list.append(df_links.iloc[i,0])
  id=df_links.iloc[i,0].split("=",1)[1] 
  s_seconds=seconds(beg[i])
  e_seconds=seconds(end[i])
  speaker_diarizer(id,i,s_seconds,e_seconds) #it is just a one video to test the model performance
 # speaker_diarizer(id,i,s_seconds,e_seconds)   Use this while working with panadas dataframe (youtube links)

# **Text Preprocessing**

In [ ]:

!pip install transformers
!pip install -q rpunct
!pip install -q language_tool_python
!pip install -q clean-text[gpl]
!pip install -q textblob
!pip install -q nlpaug
!pip install -q praw

     |████████████████████████████████| 3.5 MB 12.0 MB/s 
     |████████████████████████████████| 6.8 MB 23.7 MB/s 
     |████████████████████████████████| 895 kB 24.4 MB/s 
     |████████████████████████████████| 804.1 MB 2.5 kB/s 
     |████████████████████████████████| 213 kB 38.5 MB/s 
     |████████████████████████████████| 981 kB 36.2 MB/s 
     |████████████████████████████████| 9.9 MB 37.2 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 124 kB 47.6 MB/s 
     |████████████████████████████████| 1.7 MB 35.0 MB/s 
     |████████████████████████████████| 311 kB 50.4 MB/s 
     |████████████████████████████████| 9.7 MB 31.4 MB/s 
     |████████████████████████████████| 243 kB 50.2 MB/s 
     |████████████████████████████████| 1.1 MB 38.3 MB/s 
     |████████████████████████████████| 133 kB 51.7 MB/s 
     |████████████████████████████████| 271 kB 50.9 MB/s 
     |████████████████████████████████| 144 kB 50.7 MB/s 
     |█████████

In [ ]:
import sys
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import json
import torch
import argparse 
import gc
from tqdm import tqdm
from pathlib import Path
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# from comet_atomic2020_bart.utils import calculate_rouge, use_task_specific_params, calculate_bleu_score, trim_batch
from rpunct import RestorePuncts
import language_tool_python
import sys
import warnings
from spacy.lang.en import English
from cleantext import clean
from textblob import TextBlob
import re
import nltk
nltk.download('brown')
import nltk
nltk.download('punkt')
import nltk
from textblob import TextBlob

import pandas as pd
nltk.download('averaged_perceptron_tagger')
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from os import listdir
from os.path import isfile, join

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)


def textNormalization(text):
    processed_text=clean(text,
    fix_unicode=True,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=False,                     # lowercase text
    no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
    no_urls=  True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=True,         # replace all phone numbers with a special token
    no_numbers=False,               # replace all numbers with a special token
    no_digits=False,                # replace all digits with a special token
    no_currency_symbols=True,      # replace all currency symbols with a special token
    no_punct=False,                 # remove punctuations
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="",
    replace_with_email="<EMAIL>",
    replace_with_phone_number="<PHONE>",
    replace_with_number="<NUMBER>",
    replace_with_digit="0",
    replace_with_currency_symbol=" dollars",
    lang="en")
    return processed_text


def remove_emoji(processed_text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    processed_text=emoji_pattern.sub(r'', processed_text)
    return processed_text
def remStopWordsOur(lineIn):
    stopWords= {'i','a','and','about','an','are','as','at','be','by','com','for','from','how','in','is','it','not','of','on','or','that','the','this','to','was','what','when','where','who','will','with','the','www','your','is','am','some','you','your','I','A','And','About','An','Are','As','At','Be','By','Com','For','From','How','In','Is','It','Not','Of','On','Or','That','The','This','To','Was','What','When','Where','Who','Will','With','The','Www','Your','Is','Am','Some','You','Your','Was'}
    rmdStopWordsLn = ' '.join(w for w in lineIn.split() if w.lower() not in stopWords)
    return rmdStopWordsLn


def clean_text(text,remove_repeat_text=True, remove_patterns_text=True, is_lower=True):

# if is_lower:
#    text=text.lower()
  if remove_repeat_text:
    text = re.sub(r'(.)\1{2,}', r'\1', text) 

  text = str(text).replace("\n", " ")
  #text = re.sub(r'[^\w\s]',' ',text)
  #text = re.sub('[0-9]',"",text)
  text = re.sub(" +", " ", text)
  text = re.sub("([^\x00-\x7F])+"," ",text)
  return text


def rem_username(text):
  text = ' '.join(re.sub("([@#][A-Za-z0-9_]+)|(\w+:\/\/\S+)"," ", text).split())
  return text


#Main preprocessing block
def preprocessing_block(text):
# The default language is 'english'
  #1. Remove username
  processed_text=rem_username(text)
  #2. Text Normalization and remove URLs
  processed_text=textNormalization(processed_text)
  #3. Grametetical error correction and text cleaning
  tool = language_tool_python.LanguageTool('en')
  processed_text = tool.correct(processed_text)
  processed_text=clean_text(processed_text)
  #4 remove emoji in text
  processed_text=remove_emoji(str(processed_text))
  #5. punctuations restoring
  rpunct = RestorePuncts()
  processed_text=rpunct.punctuate(processed_text)
  #6. sentence spliting
  nlp = English()
  sbd = nlp.create_pipe('sentencizer')
  nlp.add_pipe(sbd)
  doc = nlp(str(processed_text))
  sents_list = []
  for sent in doc.sents:
     sents_list.append(sent.text)
  return sents_list


# **Dialogue Post Processing on each generated file**

# Code will iterate over each file of generated dialogues from youtube links

#1) Get rid of repeated Speakers

(Speaker 0:
Speaker 0:
Speaker 1:)
(Speaker 0:
Speaker 1: )

In [ ]:
def post_processing(path1,path2,i):
  f = open(path2+str(i), 'w')
  with open(path1) as topo_file:
      i=0
      speaker_list=[]
      for line in topo_file:
          i=i+1 
          try:
            speaker=str(line).split("Speaker",1)[1][1:2]
            speaker_list.append(speaker)
            if(speaker_list[len(speaker_list)-2]==speaker and i!=1):
              continue
            f.write(speaker+' \n')  # python will convert \n to os.linesep
          except Exception:
           # processed=preprocessing_block(line)
            f.write(line+'\n') 
  f.close() 

#2) Remove Empty Lines

In [ ]:
import re
def rem_empty_lines(path):
  with open(path, 'r+') as fd:
    lines = fd.readlines()
    fd.seek(0)
    fd.writelines(line for line in lines if line.strip())
    fd.truncate()

# 3) Concatenate 'Speaker:' Text to Speaker IDs and concatenate sentences in different lines

In [ ]:
import re
def concatenate_speaker_str(path):
  with open(path) as f:
      exampleText = f.read()

  new = ''

  for line in exampleText.split('\n'):
      if line and line[0].strip().isdigit():
          new += '\n'+'Speaker '+ line +': \n'
      if re.search('[a-zA-Z]', line) and line[0].strip().isdigit()==False :  # check if there is text
          new += line + ' '
  f = open(path, 'w')

  f.write(new) 
  f.close() 


In [ ]:
dialogues_path="/content/dialogues/"
save_path='/content/processed/myfile'
onlyfiles = [f for f in listdir(dialogues_path) if isfile(join(dialogues_path, f))]
for i in range(len(onlyfiles)):
  post_processing(dialogues_path+onlyfiles[i],save_path,i)
  rem_empty_lines(save_path+str(i))
  concatenate_speaker_str(save_path+str(i))


In [ ]:
import re
def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele  
    
    # return string  
    return str1 
        
     
def text_processing2(path):
  with open(path) as f:
      exampleText = f.read()

  new = ''

  for line in exampleText.split('\n'):
    try:
      s=preprocessing_block(line)
      string=listToString(s)
      print(string)

    except Exception as error:
   #   traceback.print_exc()
      print('..')
      string='\n'
    try:  
      speaker_id=str(line).split("Speaker",1)[1][1:2]
    except:
      speaker_id=''
      #s=['hello','world']
    #string=listToString(s)

    if line and speaker_id.isdigit():
      new+='\n'+line
    else:
      new+=string
 # f = open(path, 'w')
  print(new)
  return new
  #f.write(new) 
 #f.close() 
dialogues_path="/content/dialogues/"
save_path='/content/processed/'
text_list=[]
onlyfiles = [f for f in listdir(save_path) if isfile(join(save_path, f))]
for i in range(len(onlyfiles)):
  text=text_processing2(save_path+onlyfiles[i])
  text_list.append(text)



..


# Orignal Captions

In [ ]:
dialogues_path="/content/orignal_caption/"
save_path='/content/orignal_processed/orignal'
onlyfiles = [f for f in listdir(dialogues_path) if isfile(join(dialogues_path, f))]
for i in range(len(onlyfiles)):
  post_processing(dialogues_path+onlyfiles[i],save_path,i)
  rem_empty_lines(save_path+str(i))
  concatenate_speaker_str(save_path+str(i))


In [ ]:
import re
def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele  
    
    # return string  
    return str1 
        
     
def text_processing2(path):
  with open(path) as f:
      exampleText = f.read()

  new = ''

  for line in exampleText.split('\n'):
    try:
     # s=preprocessing_block(line)
      string=line
      print(string)

    except Exception as error:
   #   traceback.print_exc()
      print('..')
      string='\n'
    try:  
      speaker_id=str(line).split("Speaker",1)[1][1:2]
    except:
      speaker_id=''
      #s=['hello','world']
    #string=listToString(s)

    if line and speaker_id.isdigit():
      new+='\n'+line
    else:
      new+=string
 # f = open(path, 'w')
  print(new)
  return new
  #f.write(new) 
 #f.close() 
dialogues_path="/content/orignal_caption/"
save_path='/content/orignal_processed/'
orignal_text_list=[]
onlyfiles = [f for f in listdir(save_path) if isfile(join(save_path, f))]
for i in range(len(onlyfiles)):
  orignal_text=text_processing2(save_path+onlyfiles[i])
  orignal_text_list.append(orignal_text)




Speaker 0 : 
hey what's up mkbhd here so as you may have seen barack obama's been on a pretty massive book tour for his new book a promised land uh it's been a pretty great read so far for me but then youtube reached out and they're like hey do you want to ask obama anything about what he talks about in this book and so of course i said yes that would be awesome but that's that's a big moment for me like what do i ask the president if i could ask him anything so anyway what i have for you are the two questions that i chose and this is actually part of a larger youtube originals project called booktube where he answers a bunch of questions from a bunch of other creators about that book and what he wants to talk about including a really good one from mark rober about nasa and the origins of the white house science fair which i thought were really interesting so i'll link that below for you to check it out but this is a sort of extended cut of a little chat and uh i wanted to share enjoy

# Make a JSON file

In [ ]:
import json

data = {}
data['video_link'] = link_list
data['processed_text'] = text_list
data['orignal']=orignal_text_list
json_data = json.dumps(data)

In [ ]:
with open('processed.json', 'w') as outfile:
    outfile.write(json_data)